In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from datetime import datetime
from pandas import Series
df=pd.read_csv("C:/Users/harshal/Downloads/Electric_Production.csv")
df1=df.copy()
df.head()

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df["DATE"]=pd.to_datetime(df["DATE"])
df = df.set_index(df["DATE"])

#df.head()

df.tail()

In [ ]:
df["Days"]=df.DATE.dt.day
df["Month"]=df.DATE.dt.month
df["Year"]=df.DATE.dt.year
df["Day of week"]=df.DATE.dt.dayofweek
df["Week"]=df.DATE.dt.week
df.head()
def weekend_applyer(row):
    if row.dayofweek==5 or row.dayofweek==6:
        return 1
    else:
        return 0
temp2=df["DATE"].apply(weekend_applyer)
df["Weekend"]=temp2
df.head()

In [ ]:
#date vs electric production in general
df.index=df.DATE
plt.figure(figsize=(15,5))
plt.plot(df.index, df.IPG2211A2N)
plt.xlabel("Year")
plt.ylabel("Electric Production")
plt.title("Electric Production and Year")
print(df.index)

In [ ]:
#Year vs electric production
plt.figure(figsize=(10,8))
df.groupby("Year")["IPG2211A2N"].mean().plot(kind="bar")
#2018 extremely high

In [ ]:
#month vs electric production
plt.figure(figsize=(10,8))
df.groupby("Month")["IPG2211A2N"].mean().plot(kind="bar")
#no noticeable trend

In [ ]:
#week number vs electric production
plt.figure(figsize=(10,8))
df.groupby("Week")["IPG2211A2N"].mean().plot(kind="bar")
#Production is stable across weeks

In [ ]:
#day of week vs electric production
plt.figure(figsize=(10,8))
df.groupby("Day of week")["IPG2211A2N"].mean().plot(kind="bar")
#Production is stable across the days

In [ ]:
#Weekend vs electric production
plt.figure(figsize=(10,8))
df.groupby("Weekend")["IPG2211A2N"].mean().plot(kind="bar")
#Production is stable regardless of weekday or weekend

In [ ]:
df.groupby(["Year", "Month"])["IPG2211A2N"].mean().plot(kind="bar")

In [ ]:
month=df.resample("M").mean()
day=df.resample("D").mean()
year=df.resample("A").mean()
week=df.resample("W").mean()
newday=pd.DataFrame(day)
newday=newday.dropna()
newweek=pd.DataFrame(week)
newweek=newweek.dropna()
fig,ax=plt.subplots(4,1)
month.IPG2211A2N.plot(fontsize=14, title="Monthly", ax=ax[0])
newday.IPG2211A2N.plot(fontsize=14, title="Daily", ax=ax[1])
year.IPG2211A2N.plot(fontsize=14, title="Yearly", ax=ax[2])
newweek.IPG2211A2N.plot(fontsize=14, title="Weekly", ax=ax[3])
plt.ylabel("Electric Production")

In [ ]:
#resample month--month plot has least amount of noise
df.Timestamp=pd.to_datetime(df.DATE)
df.index=df.Timestamp
df=df.resample("M").mean()

In [ ]:
print(len(df.index)*0.7)
df.head()
df[df.index.day==31]

In [ ]:
#resample months
train=df.loc["1985-01-01":"2008-01-01"]
valid=df.loc["2008-01-01":]
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
#1. Naive approach
arr=np.asarray(train.IPG2211A2N)
y_hat=valid.copy()
y_hat["naive"]=arr[len(arr)-1]
#resample months
train=df.loc["1985-01-01":"2008-01-01"]
valid=df.loc["2008-01-01":]
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat['naive'].plot()
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["naive"]))
rmse

In [ ]:
#2. Simple Average
y_hat=valid.copy()
y_hat['simple average']=(train.IPG2211A2N).mean()
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["simple average"].plot(figsize=(15,8), fontsize=14, label="simple average")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["simple average"]))
rmse

In [ ]:
#3. Moving average
y_hat=valid.copy()
y_hat["moving average forecast"]=train.IPG2211A2N.rolling(10).mean().iloc[-1]
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["moving average forecast"].plot(figsize=(15,8), fontsize=14, label="moving average with last 10 values")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")
#print(y_hat["moving average forecast"])

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["moving average forecast"]))
rmse

In [ ]:
#3. Moving average
y_hat=valid.copy()
y_hat["moving average forecast"]=train.IPG2211A2N.rolling(20).mean().iloc[-1]
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["moving average forecast"].plot(figsize=(15,8), fontsize=14, label="moving average with last 20 values")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")
#print(y_hat["moving average forecast"])

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["moving average forecast"]))
rmse

In [ ]:
#3. Moving average
y_hat=valid.copy()
y_hat["moving average forecast"]=train.IPG2211A2N.rolling(30).mean().iloc[-1]
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["moving average forecast"].plot(figsize=(15,8), fontsize=14, label="moving average with last 30 values")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")
#print(y_hat["moving average forecast"])

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["moving average forecast"]))
rmse

In [ ]:
#Moving average forecast works best for 20 values when it comes to this particular dataset.

In [ ]:
#4. Simple Exponential Smoothing
from statsmodels.tsa.api import SimpleExpSmoothing
y_hat=valid.copy()
fit=SimpleExpSmoothing(np.asarray(train.IPG2211A2N)).fit(smoothing_level=0.3, optimized=False)
y_hat["SES"]=fit.forecast(len(valid))
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["SES"].plot(figsize=(15,8), fontsize=14, label="Simple Exp Smoothing")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["SES"]))
rmse

In [ ]:
#5. Holt's Linear Model
#First, decompose the time series
import statsmodels.api as sm
sm.tsa.seasonal_decompose(train.IPG2211A2N).plot()
result=sm.tsa.stattools.adfuller(train.IPG2211A2N)
plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import Holt
y_hat=valid.copy()
fit1=Holt(np.asarray(train.IPG2211A2N)).fit(smoothing_level=0.01, smoothing_slope=0.01)
y_hat["Holt's Linear Model"]=fit1.forecast(len(valid))
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["Holt's Linear Model"].plot(figsize=(15,8), fontsize=14, label="Holt's Linear Model")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["Holt's Linear Model"]))
rmse

In [ ]:
#6. Holt's Winter Method
from statsmodels.tsa.holtwinters import ExponentialSmoothing
y_hat=valid.copy()
fit2=ExponentialSmoothing(np.asarray(train.IPG2211A2N), trend="add", seasonal="add", seasonal_periods=12).fit()
y_hat["Holt Winter"]=fit2.forecast(len(valid))
train.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="train")
valid.IPG2211A2N.plot(figsize=(15,8), fontsize=14, label="Valid")
y_hat["Holt Winter"].plot(figsize=(15,8), fontsize=14, label="Holt Winter")
plt.xlabel("Date")
plt.ylabel("Electric production")
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(valid.IPG2211A2N, y_hat["Holt Winter"]))
rmse

In [ ]:
#Dickey fuller test to test for stationaritty
#Function
#Steps
#1. find rolling mean and std
#2. Plot them
#3. convert timeseries to a Series(pd.Series) then call function
from statsmodels.tsa.api import adfuller

def test_stationarity(timeseries):
    roll_mean=timeseries.rolling(12).mean()  #last 12 values because 12 months in a year
    roll_std=timeseries.rolling(12).std()
    
    rollmean=plt.plot(roll_mean, label="Rolling mean")
    rollstd=plt.plot(roll_std, label="Rolling std")
    original=plt.plot(timeseries, label="Original")
    
    plt.legend(loc="best")
    
    print("Results of Dickey Fuller test")
    dftest=adfuller(timeseries, autolag="AIC")
    new_series=pd.Series(dftest[0:4], index=["Test stat", "P-Value", "# of Lags", "# of Obs"])
    for key, val in dftest[4].items():
        new_series["Critical values %s" %key]=val
    print(new_series)
    
test_stationarity(train.IPG2211A2N)
#Test statistic not less than critical values, so we need to eliminate trend and seasonality

In [ ]:
#Methods to deal with trend--taking the log of the original(then graph), taking the moving average, and then subtracting the two.
#Then check test stat again
train_log=np.log(train.IPG2211A2N)
valid_log=np.log(valid.IPG2211A2N)
moving_avg=train_log.rolling(12).mean()
plt.plot(train_log, label="")
plt.plot(moving_avg)

In [ ]:
log_diff=train_log-moving_avg
log_diff=log_diff.dropna()
test_stationarity(log_diff)

In [ ]:
#taking care of both trend and seasonality
new_train_log=train_log-train_log.shift(1)
new_train_log.dropna(inplace=True)
test_stationarity(new_train_log)

In [ ]:
from statsmodels.tsa.api import seasonal_decompose
decompose=seasonal_decompose(train_log, freq=12)
residual=decompose.resid
seasonal=decompose.seasonal
trend=decompose.trend
plt.figure(figsize=(15,8))
plt.subplot(411)
plt.plot(train_log, label="Original")
plt.legend(loc="best")
plt.subplot(412)
plt.plot(trend, label="Trend")
plt.legend(loc="best")
plt.subplot(413)
plt.plot(seasonal, label="Seasonal")
plt.legend(loc="best")
plt.subplot(414)
plt.plot(residual, label="Residual")
plt.legend(loc="best")

In [ ]:
#decompose residual's trend and stationarity
train_log_decompose = residual
train_log_decompose.dropna(inplace=True)
test_stationarity(train_log_decompose)

In [ ]:
#Plot ACF, PACF
from statsmodels.tsa.stattools import acf, pacf #plotted after differencing
lag_acf=acf(new_train_log, nlags=25)
lag_pacf=pacf(new_train_log, nlags=25, method="ols")
#plotting
plt.plot(lag_acf)
plt.axhline(y=0, linestyle="--", color="grey")
plt.axhline(y=1.96*(1/np.sqrt(len(valid))), linestyle="--", color="grey")
plt.axhline(y=-1.96*(1/np.sqrt(len(valid))), linestyle="--", color="grey")
plt.title("Autocorrelation Function")
plt.show()
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle="--", color="grey")
plt.axhline(y=1.96*(1/np.sqrt(len(valid))), linestyle="--", color="grey")
plt.axhline(y=-1.96*(1/np.sqrt(len(valid))), linestyle="--", color="grey")
plt.title("Partial Autocorrelation Function")
plt.show()

In [ ]:
#ARIMA model
#AR model
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(train_log, order=(2,1,0))
results_AR=model.fit(disp=-1)
plt.plot(new_train_log, label="Graph post differencing")
plt.plot(results_AR.fittedvalues, label="Prediction using AR model")
plt.legend(loc="best")
#print("RMSE: %.4f" %(np.sqrt(sum(results_AR.fittedvalues-train_log))/(len(train_log))))
#print(new_train_log)

In [ ]:
#MA model
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(train_log, order=(0,1,3))
results_MA=model.fit(disp=-1)
plt.plot(new_train_log, label="Graph post differencing")
plt.plot(results_MA.fittedvalues, label="Prediction using MA mode")
plt.legend(loc="best")

In [ ]:
#Combined model
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(train_log, order=(2,1,3))
results_combined=model.fit(disp=-1)
plt.plot(new_train_log, label="Graph post differencing")
plt.plot(results_combined.fittedvalues, label="Combining AR and MA graphs")
plt.legend(loc="best")

In [ ]:
#Bring it back to original scale
#Steps
#1.converting to a series(results_combined)
#2. take the cumsum()
#3. put the cumsum values into another series, and begin with the 0th index
#4. add cumsum() to the series
#5. take exp
predictions_diff=pd.Series(results_combined.fittedvalues)
#print(predictions_diff)
predictions_diff_cumsum=predictions_diff.cumsum()
combined_model=pd.Series(train_log.iloc[0], index=train_log.index)
combined_model=combined_model.add(predictions_diff_cumsum, fill_value=0)
exp_model=np.exp(combined_model)
plt.plot
plt.plot(exp_model, label="Predictions")
plt.plot(train.IPG2211A2N, label="Train")
#plt.plot(valid.IPG2211A2N, label="Valid")
plt.title("RMSE: %.4f" %(np.sqrt(sum((combined_model-train_log)**2))/(len(train_log))))
plt.legend(loc="best")

In [ ]:
#7. Using Sarimax(seasonal ARIMA)
y_hat=valid.copy()
from statsmodels.tsa.statespace.sarimax import SARIMAX
model=SARIMAX(train.IPG2211A2N, seasonal_order=(0,1,1,12), order=(1,1,4)).fit()
y_hat["SARIMAX"]=model.forecast(len(valid))
plt.figure(figsize=(15,8))
plt.plot(y_hat["SARIMAX"], label="SARIMAX")
plt.plot(train.IPG2211A2N, label="Train")
plt.plot(valid.IPG2211A2N, label="Valid")
#plt.figure(figsize=(15,8))
plt.legend(loc="best")

In [ ]:
rmse=np.sqrt(mean_squared_error(y_hat["SARIMAX"], valid.IPG2211A2N))
rmse